In [1]:
import numpy as np
from math import sqrt, fabs
import pandas as pd

In [2]:
#Формула Герона для нахождения корня


def teylor_sqrt(c):
    if c < 1:
        x = 1
    else:
        x = c
    y = 0
    k = 1
    while abs(x-y) > 1e-10:
        if k%2 == 1:
            y = (x+c/x)/2
        else:
            x = (y+c/y)/2
        k += 1
    if k%2 == 1:
        return x
    else:
        return y

In [3]:
#eps, которуя я использую


eps = 1e-3

In [4]:
class Matrix:
    
    A = [[]]
    def __init__(self, dim):
        self.dim1 = dim[0]
        self.dim2 = dim[1]
        self.A = [[0]*self.dim2 for i in range(self.dim1)]
        
    def __getitem__(self, i):
        return self.A[i]
    
    def __eq__(self, B):
        assert(self.dim1 == B.dim1 and self.dim2 == B.dim2)
        for i in range(self.dim1):
            for j in range(self.dim2):
                if fabs(self.A[i][j] - B[i][j]) > 1e-9:
                    return False
        return True
    
    def dim(self):
        return (self.dim1,self.dim2)
    
    def __add__(self, B):
        assert((self.dim1,self.dim2) == B.dim())
        result = Matrix((self.dim1,self.dim2))
        for i in range(self.dim1):
            for j in range(self.dim2):
                result[i][j] = self.A[i][j] + B[i][j]
        return result
    
    def __sub__(self, B):
        assert((self.dim1,self.dim2) == B.dim())
        result = Matrix((self.dim1,self.dim2))
        for i in range(self.dim1):
            for j in range(self.dim2):
                result[i][j] = self.A[i][j] - B[i][j]
        return result
    
    def __mul__(self, B):
        if(type(B) == int or type(B) == float):
            result = Matrix((self.dim1,self.dim2))
            for i in range(self.dim1):
                for j in range(self.dim2):
                    result[i][j] = self.A[i][j]*B
            return result
        else:
            assert(self.dim2 == B.dim1)
            result = Matrix((self.dim1,B.dim2))
            for i in range(self.dim1):
                for j in range(B.dim2):
                    for k in range(B.dim1):
                        result[i][j] += self.A[i][k]*B[k][j]
            return result
    
    def __truediv__(self,a):
        result = Matrix((self.dim1,self.dim2))
        for i in range(self.dim1):
            for j in range(self.dim2):
                result[i][j] = self.A[i][j]/a
        return result
    
    def transpose(self):
        result = Matrix((self.dim2,self.dim1))
        for i in range(self.dim1):
            for j in range(self.dim2):
                result[j][i] = self.A[i][j]
        return result
    
    def norm(self):
        result = 0
        for i in range(self.dim1):
            for j in range(self.dim2):
                result += self.A[i][j]*self.A[i][j]
        return teylor_sqrt(result)
    
    def norm_abs(self):
        result = 0
        for i in range(self.dim1):
            for j in range(self.dim2):
                result += ab(self.A[i][j])
        return result
    
    def iseye(self):
        assert(self.dim1 == self.dim2)
        for i in range(self.dim1):
            for j in range(self.dim1):
                if i == j:
                    self.A[i][j] = 1
                else:
                    self.A[i][j] = 0
                    
    def absolut(self):
        for i in range(self.dim1):
            for j in range(self.dim2):
                self.A[i][j] = abs(self.A[i][j])
        
    def det(self):
        assert(self.dim1 == self.dim2)
        result = Matrix((self.dim1,self.dim2))
        for i in range(self.dim1):
            for j in range(self.dim2):
                result[i][j] = self.A[i][j]
        max_A = result[0][0]
        column = 0
        for i in range(self.dim1-1):
            max_A = result[i][i]
            column = i
            for j in range(i,self.dim1):
                if result[j][i] > max_A:
                    max_A = result[i][j]
                    column = j
            for j in range(i,self.dim1):
                result[i][j], result[column][j] = result[column][j], result[i][j]
            #
            for v in range(i+1,self.dim1):
                koef = -(result[v][i]/result[i][i])
                for j in range(i, self.dim1):
                    result[v][j] += result[i][j]*koef
        Res = 1
        for i in range(self.dim1):
            Res *= result[i][i]
        return Res
    
    def diag(self):
        assert(self.dim1 == self.dim2)
        for i in range(self.dim1):
            S = sum(abs(A[i][j]) for j in range(self.dim2) if j != i)
            if fabs(A[i][i]) < S:
                return False
        return True
    
    def Edin(self):
        for i in range(self.dim1):
            for j in range(self.dim2):
                self.A[i][j] = 1
            
    def __str__(self):
        return str(self.A)

In [5]:
def simple_iteration(A,b):
    A1 = A
    b1 = b
    T = A.transpose()
    E = Matrix((A.dim1,A.dim1))
    E.iseye()
    if not A == T or (E-A/A.norm()).norm() >= 1:
        A = T*A
        b = T*(b.transpose())
        b = b.transpose()
    S = []
    k = 0
    my = 1/A.norm()
    c = b*my
    c = c.transpose()
    E = Matrix((A.dim1,A.dim1))
    E.iseye()
    B = E - A*my
    norm_B = B.norm()
    #if norm_B >= 1:
    #    A = A.transpose()*A
    #    b = A.transpose()*b.transpose()
    #    my = 1/A.norm()
    #    c = b*my
    #    E = Matrix((A.dim1,A.dim1))
    #    E.iseye()
    #    B = E - A*my
    #    norm_B = B.norm()
    S.append(c)
    while True:
        S.append(B*S[k] + c)
        k += 1
        if norm_B >= 1:
            N = ((A1*S[k]).transpose() - b).norm()
        else:
            N = norm_B/(1-norm_B)*(S[k] - S[k-1]).norm()
        if N < eps or k >= 100000:
            break
    return S[k]

In [6]:
#Метод Зейделя


def Seidel(A,b):
    T = A.transpose()
    if not A == T or not A.diag():
        A = T*A
        b = T*(b.transpose())
        b = b.transpose()
    Cij = Matrix((A.dim1, A.dim2))
    D = Matrix((b.dim1, b.dim2))
    x = []
    k = 0
    for i in range(A.dim1):
        for j in range(A.dim2):
            if j != i:
                Cij[i][j] = -A[i][j]/A[i][i]
            else:
                Cij[i][j] = 0
        D[0][i] = b[0][i]/A[i][i]
    while True:
        if len(x) == 0:
            x.append(D)
        f = Matrix((b.dim1,b.dim2))
        l = 0
        for i in range(b.dim2):
            for j in range(l):
                #if j != i:
                f[0][i] += Cij[i][j]*f[0][j]
            for j in range(l, b.dim2):
                #if j != i:
                f[0][i] += Cij[i][j]*x[k][0][j]
            l += 1
            f[0][i] += D[0][i]
        x.append(f)
        k += 1
        N = ((A*(x[k].transpose())).transpose() - b).norm()
        if N < eps or k >= 100000:
            break
    return x[k]

In [7]:
def Gauss(A,b):
    #Прямой ход Гаусса
    for i in range(A.dim1-1):
        max_A = A[i][i]
        column = i
        for j in range(i,A.dim1):
            if A[j][i] > max_A:
                max_A = A[i][j]
                column = j
        for j in range(i,A.dim1):
            A[i][j], A[column][j] = A[column][j], A[i][j]
        b[0][i], b[0][column] = b[0][column], b[0][i]
        for v in range(i+1,A.dim1):
            koef = -(A[v][i]/A[i][i])
            for j in range(i, A.dim1):
                A[v][j] += A[i][j]*koef
            b[0][v] += b[0][i]*koef
    #обратный ход Гаусса
    x = Matrix((b.dim1,b.dim2))
    for i in range(A.dim1-1, -1, -1):
        Sum = 0
        for j in range(A.dim1 - i - 1):
            Sum += A[i][A.dim1-j-1]*x[0][A.dim1-j-1]
        x[0][i] += (b[0][i]-Sum)/A[i][i]
    return x

In [8]:
#Функция определения знака. Она используется в QR разложении(формула из википедии)


sign = lambda x: 1 if x>-1e-14 else -1

In [9]:
#QR разложение и обратный ход Гаусса - метод Отражений


def QR(A):
    R = Matrix((A.dim1,A.dim1))
    Q = Matrix((A.dim1,A.dim1))
    for i in range(A.dim1):
        for j in range(A.dim1):
            R[i][j] = A[i][j]
    for k in range(A.dim1-1):
        z = Matrix((A.dim1-k,1))
        x = Matrix((A.dim1-k,1))
        z[0][0] = 1
        for i in range(k,A.dim1):
            x[i-k][0] = R[i][k]  
        alpha = x.norm()*(-sign(x[0][0]))    
        u = x - z*alpha
        v = u/u.norm()
        Qk = Matrix((A.dim1-k,A.dim1-k))
        Qk.iseye()
        Qk = Qk - v*v.transpose()*2
        Qt = Matrix((A.dim1,A.dim1))
        for i in range(A.dim1):
            for j in range(A.dim1):
                if i < k or j < k:
                    if i == j:
                        Qt[i][j] = 1
                    else:
                        Qt[i][j] = 0
                else:
                    Qt[i][j] = Qk[i-k][j-k]
        if k == 0:
            Q = Qt
            R = Qt*A
        else:
            Q = Qt*Q
            R = Qt*R
    return Q.transpose(), R

def GasGolder(A,b):
    Q, R = QR(A)
    b = Q.transpose()*b.transpose()
    n = A.dim1
    x = Matrix((n,1))
    x[n-1][0] = b[n-1][0]/R[n-1][n-1]
    for i in reversed(range(n-1)):
        Sum = 0
        for j in range(i,n):
            Sum += R[i][j]*x[j][0]
        x[i][0] += (b[i][0]-Sum)/R[i][i]
    return x

In [10]:
#0 задача
A1 = [[0,2,3],[1,2,4],[4,5,6]]
b1 = [[13,17,32]]
x0 = np.linalg.solve(np.array(A1),np.array(b1).transpose())
n = 3
A = Matrix((n,n))
b = Matrix((1,n))
for i in range(A.dim1):
    for j in range(A.dim2):
        A[i][j] = A1[i][j]
    b[0][i] = b1[0][i]
SI0 = simple_iteration(A,b)
SE0 = Seidel(A,b)
GAS0 = GasGolder(A,b)
GAU0 = Gauss(A,b)

In [11]:
#1 задача
A1 = [[18,1,1],[1,20,1],[1,1,22]]
b1 = [[20,22,24]]
x1 = np.linalg.solve(np.array(A1),np.array(b1).transpose())
n = 3
A = Matrix((n,n))
b = Matrix((1,n))
for i in range(A.dim1):
    for j in range(A.dim2):
        A[i][j] = A1[i][j]
    b[0][i] = b1[0][i]
SI1 = simple_iteration(A,b)
SE1 = Seidel(A,b)
GAS1 = GasGolder(A,b)
GAU1 = Gauss(A,b)

In [12]:
#2 задача
A1 = [[-18,1,1],[1,-20,1],[1,1,-22]]
b1 = [[-20,-22,-24]]
x2 = np.linalg.solve(np.array(A1),np.array(b1).transpose())
n = 3
A = Matrix((n,n))
b = Matrix((1,n))
for i in range(A.dim1):
    for j in range(A.dim2):
        A[i][j] = A1[i][j]
    b[0][i] = b1[0][i]
SI2 = simple_iteration(A,b)
SE2 = Seidel(A,b)
GAS2 = GasGolder(A,b)
GAU2 = Gauss(A,b)

In [13]:
#3 задача
A1 = [[-18,19,20],[21,-20,17],[20,21,-22]]
b1 = [[20,22,24]]
x3 = np.linalg.solve(np.array(A1),np.array(b1).transpose())
n = 3
A = Matrix((n,n))
b = Matrix((1,n))
for i in range(A.dim1):
    for j in range(A.dim2):
        A[i][j] = A1[i][j]
    b[0][i] = b1[0][i]
SI3 = simple_iteration(A,b)
SE3 = Seidel(A,b)
GAS3 = GasGolder(A,b)
GAU3 = Gauss(A,b)

In [14]:
#4 задача
A1 = [[18,17,17],[17,20,17],[17,17,21]]
b1 = [[20,22,24]]
x4 = np.linalg.solve(np.array(A1),np.array(b1).transpose())
n = 3
A = Matrix((n,n))
b = Matrix((1,n))
for i in range(A.dim1):
    for j in range(A.dim2):
        A[i][j] = A1[i][j]
    b[0][i] = b1[0][i]
SI4 = simple_iteration(A,b)
SE4 = Seidel(A,b)
GAS4 = GasGolder(A,b)
GAU4 = Gauss(A,b)

In [15]:
#5 задача
#ниже можно поменять размерность матрицы A
n = 5
#
A1 = np.eye(n,n)
T1 = np.eye(n,n)
b1 = np.eye(1,n)
A = Matrix((n,n))
T = Matrix((n,n))
b = Matrix((1,n))
A.iseye()
for i in range(n):
    for j in range(n):
        if j > i:
            A1[i][j] = -1
            A[i][j] = -1
            T[i][j] = -1
        else:
            T[i][j] = 1
            T1[i][j] = 1
    b[0][i] = -1
    b1[0][i] = -1
b1[0][n-1] = 1
b[0][n-1] = 1
A = A + T*(eps*16)
x5 = np.linalg.solve(np.array(A1),np.array(b1).transpose())
SI5 = simple_iteration(A,b)
SE5 = Seidel(A,b)
GAS5 = GasGolder(A,b)
GAU5 = Gauss(A,b)

In [16]:
SI11,SI22,SI33,SI44,SI55 = [], [], [], [], []
SE11,SE22,SE33,SE44,SE55 = [], [], [], [], []
GAU11,GAU22,GAU33,GAU44,GAU55 = [], [], [], [], []
GAS11,GAS22,GAS33,GAS44,GAS55 = [], [], [], [], []
for i in range(3):
    x1[i][0],x2[i][0],x3[i][0],x4[i][0] = x1[i][0],x2[i][0],x3[i][0],x4[i][0]
    SI11.append(SI1[i][0])
    SI22.append(SI2[i][0])
    SI33.append(SI3[i][0])
    SI44.append(SI4[i][0])
    SI55.append(SI5[i][0])
    #
    SE11.append(SE1[0][i])
    SE22.append(SE2[0][i])
    SE33.append(SE3[0][i])
    SE44.append(SE4[0][i])
    SE55.append(SE5[0][i])
    #
    GAU11.append(GAU1[0][i])
    GAU22.append(GAU2[0][i])
    GAU33.append(GAU3[0][i])
    GAU44.append(GAU4[0][i])
    GAU55.append(GAU5[0][i])
    #
    GAS11.append(GAS1[i][0])
    GAS22.append(GAS2[i][0])
    GAS33.append(GAS3[i][0])
    GAS44.append(GAS4[i][0])
    GAS55.append(GAS5[i][0])
print('Первое')
print('Первое решение numpy: ', x1.transpose())
print('Использовал eps = 10^(-3)')
print('Решение с помошью метода МПИ: ', SI11)
print('Решение с помошью метода Зейделя: ', SE11)
print('Решение с помошью метода Гаусса: ', GAU11)
print('Решение с помошью метода Хасхолдера: ', GAS11)
print('абслютная разность с помошью метода МПИ: ', [abs(SI11[i] - x1[i]) for i in range(3)])
print('абслютная разность с помошью метода Зейделя: ', [abs(SE11[i] - x1[i]) for i in range(3)])
print('абслютная разность с помошью метода Гаусса: ', [abs(GAU11[i] - x1[i]) for i in range(3)])
print('абслютная разность с помошью метода Хасхолдера: ', [abs(GAS11[i] - x1[i]) for i in range(3)])
print('\n')
print('Второе')
print('Второе решение numpy: ', x2.transpose())
print('Использовал eps = 10^(-3)')
print('Решение с помошью метода МПИ: ', SI22)
print('Решение с помошью метода Зейделя: ', SE22)
print('Решение с помошью метода Гаусса: ', GAU22)
print('Решение с помошью метода Хасхолдера: ', GAS22)
print('абслютная разность с помошью метода МПИ: ', [abs(SI22[i] - x2[i]) for i in range(3)])
print('абслютная разность с помошью метода Зейделя: ', [abs(SE22[i] - x2[i]) for i in range(3)])
print('абслютная разность с помошью метода Гаусса: ', [abs(GAU22[i] - x2[i]) for i in range(3)])
print('абслютная разность с помошью метода Хасхолдера: ', [abs(GAS22[i] - x2[i]) for i in range(3)])
print('\n')
print('Третье')
print('Третье решение numpy: ', x3.transpose())
print('Использовал eps = 10^(-3)')
print('Решение с помошью метода МПИ: ', SI33)
print('Решение с помошью метода Зейделя: ', SE33)
print('Решение с помошью метода Гаусса: ', GAU33)
print('Решение с помошью метода Хасхолдера: ', GAS33)
print('абслютная разность с помошью метода МПИ: ', [abs(SI33[i] - x3[i]) for i in range(3)])
print('абслютная разность с помошью метода Зейделя: ', [abs(SE33[i] - x3[i]) for i in range(3)])
print('абслютная разность с помошью метода Гаусса: ', [abs(GAU33[i] - x3[i]) for i in range(3)])
print('абслютная разность с помошью метода Хасхолдера: ', [abs(GAS33[i] - x3[i]) for i in range(3)])
print('\n')
print('Четвёртое')
print('Четвёртое решение numpy: ', x4.transpose())
print('Использовал eps = 10^(-3)')
print('Решение с помошью метода МПИ: ', SI44)
print('Решение с помошью метода Зейделя: ', SE44)
print('Решение с помошью метода Гаусса: ', GAU44)
print('Решение с помошью метода Хасхолдера: ', GAS44)
print('абслютная разность с помошью метода МПИ: ', [abs(SI44[i] - x4[i]) for i in range(3)])
print('абслютная разность с помошью метода Зейделя: ', [abs(SE44[i] - x4[i]) for i in range(3)])
print('абслютная разность с помошью метода Гаусса: ', [abs(GAU44[i] - x4[i]) for i in range(3)])
print('абслютная разность с помошью метода Хасхолдера: ', [abs(GAS44[i] - x4[i]) for i in range(3)])
print('\n')
print('Пятое')
print(f'при n = {n}')
print('Пятое решение numpy: ', x5.transpose())
print('Использовал eps = 10^(-3)')
print('Решение с помошью метода МПИ: ', SI5)
print('Решение с помошью метода Зейделя: ', SE5)
print('Решение с помошью метода Гаусса: ', GAU5)
print('Решение с помошью метода Хасхолдера: ', GAS5)
print('абслютная разность с помошью метода МПИ: ', [abs(SI55[i] - x5[i]) for i in range(3)])
print('абслютная разность с помошью метода Зейделя: ', [abs(SE55[i] - x5[i]) for i in range(3)])
print('абслютная разность с помошью метода Гаусса: ', [abs(GAU55[i] - x5[i]) for i in range(3)])
print('абслютная разность с помошью метода Хасхолдера: ', [abs(GAS55[i] - x5[i]) for i in range(3)])

Первое
Первое решение numpy:  [[1. 1. 1.]]
Использовал eps = 10^(-3)
Решение с помошью метода МПИ:  [0.9997754808453181, 1.0000118228289496, 1.0000482865102285]
Решение с помошью метода Зейделя:  [1.0000027258108712, 1.0000001900835627, 0.9999998674593438]
Решение с помошью метода Гаусса:  [1.0, 1.0, 1.0000000000000002]
Решение с помошью метода Хасхолдера:  [0.9999999999999998, 1.0000000000000002, 1.0000000000000002]
абслютная разность с помошью метода МПИ:  [array([0.00022452]), array([1.18228289e-05]), array([4.82865102e-05])]
абслютная разность с помошью метода Зейделя:  [array([2.72581087e-06]), array([1.90083562e-07]), array([1.32540656e-07])]
абслютная разность с помошью метода Гаусса:  [array([0.]), array([2.22044605e-16]), array([0.])]
абслютная разность с помошью метода Хасхолдера:  [array([2.22044605e-16]), array([0.]), array([0.])]


Второе
Второе решение numpy:  [[1.24586409 1.22244846 1.20310512]]
Использовал eps = 10^(-3)
Решение с помошью метода МПИ:  [1.2455424872708258

In [17]:
A1 = [[0,2,3],[1,2,4],[4,5,6]]
b1 = [[13,17,32]]
x4 = np.linalg.solve(np.array(A1),np.array(b1).transpose())
n = 3
A = Matrix((n,n))
b = Matrix((1,n))
for i in range(A.dim1):
    for j in range(A.dim2):
        A[i][j] = A1[i][j]
    b[0][i] = b1[0][i]
SI4 = simple_iteration(A,b)
SE4 = Seidel(A,b)
GAS4 = GasGolder(A,b)
GAU4 = Gauss(A,b)
print(x4.transpose())

[[1. 2. 3.]]


In [18]:
print(SI4)
print(SE4)
print(GAS4)
print(GAU4)

[[0.999999999999931], [2.0000000000001545], [2.99999999999992]]
[[1.001067691042504, 1.9977285158809979, 3.0011483598696067]]
[[1.0000000000000004], [1.999999999999998], [3.000000000000001]]
[[1.0, 2.0, 3.0]]
